In [2]:
'''
This script
* Loads documents as aggregation of tweets stored in a MongoDB collection
* Cleans up the documents
* Creates a dictionary and corpus that can be used to train an LDA model
* Training of the LDA model is not included but follows:
  lda = models.LdaModel(corpus, id2word=dictionary, num_topics=100, passes=100)
Author: Alex Perrier
Python 2.7
'''

import langid
import nltk
import re
import time
from collections import defaultdict
from configparser import ConfigParser
from gensim import corpora, models, similarities
from nltk.tokenize import RegexpTokenizer
from pymongo import MongoClient
from string import digits


#def filter_lang(lang, documents):
#    doclang = [  langid.classify(doc) for doc in documents ]
#    return [documents[k] for k in range(len(documents)) if doclang[k][0] == lang]

# connect to the MongoDB
#client      = MongoClient()
#db          = client['twitter']

# Load documents and followers from db
# Filter out non-english timelines and TL with less than 2 tweets
#documents    = [tw['raw_text'] for tw in db.tweets.find()
#                    if ('lang' in tw.keys()) and (tw['lang'] in ('en','und'))
#                        and ('n_tweets' in tw.keys()) and (tw['n_tweets'] > 2) ]

#  Filter non english documents
#documents = filter_lang('en', documents)
#print("We have " + str(len(documents)) + " documents in english ")

# Remove urls
#documents = [re.sub(r"(?:\@|http?\://)\S+", "", doc)
#                for doc in documents ]

# Remove documents with less 100 words (some timeline are only composed of URLs)
#documents = [doc for doc in documents if len(doc) > 100]
data = [line.rstrip() for line in open('data.txt')]
import string
for i in range(len(data)):
    for c in string.punctuation:
        data[i]= data[i].replace(c,"")       
documents=data[:100]
# tokenize
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
documents = [ tokenizer.tokenize(doc.lower()) for doc in documents ]

# Remove stop words
stoplist_tw=['amp','get','got','hey','hmm','hoo','hop','iep','let','ooo','par',
            'pdt','pln','pst','wha','yep','yer','aest','didn','nzdt','via',
            'one','com','new','like','great','make','top','awesome','best',
            'good','wow','yes','say','yay','would','thanks','thank','going',
            'new','use','should','could','best','really','see','want','nice',
            'while','know']

unigrams = [ w for doc in documents for w in doc if len(w)==1]
bigrams  = [ w for doc in documents for w in doc if len(w)==2]

stoplist  = set(nltk.corpus.stopwords.words("english") + stoplist_tw
                + unigrams + bigrams)
documents = [[token for token in doc if token not in stoplist]
                for doc in documents]

# rm numbers only words
documents = [ [token for token in doc if len(token.strip(digits)) == len(token)]
                for doc in documents ]

# Lammetization
# This did not add coherence ot the model and obfuscates interpretability of the
# Topics. It was not used in the final model.
#   from nltk.stem import WordNetLemmatizer
#   lmtzr = WordNetLemmatizer()
#   documents=[[lmtzr.lemmatize(token) for token in doc ] for doc in documents]

# Remove words that only occur once
token_frequency = defaultdict(int)

# count all token
for doc in documents:
    for token in doc:
        token_frequency[token] += 1

# keep words that occur more than once
documents = [ [token for token in doc if token_frequency[token] > 1]
                for doc in documents  ]

# Sort words in documents
for doc in documents:
    doc.sort()

# Build a dictionary where for each document each word has its own id
dictionary = corpora.Dictionary(documents)
dictionary.compactify()
# and save the dictionary for future use
dictionary.save('JamesT.dict')

# We now have a dictionary with 26652 unique tokens
print(dictionary)

# Build the corpus: vectors with occurence of each word for each document
# convert tokenized documents to vectors
corpus = [dictionary.doc2bow(doc) for doc in documents]

# and save in Market Matrix format
corpora.MmCorpus.serialize('JamesT.mm', corpus)
# this corpus can be loaded with corpus = corpora.MmCorpus('alexip_followers.mm')


Dictionary(70 unique tokens: ['extra', 'surface', 'brilliant', 'permanent', 'tip']...)


In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)
from gensim import corpora, models, similarities, matutils
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

In [5]:
# Define KL function
def sym_kl(p,q):
    return np.sum([stats.entropy(p,q),stats.entropy(q,p)])

In [6]:
my_corpus=corpus

In [8]:
l = np.array([sum(cnt for _, cnt in doc) for doc in my_corpus])
def arun(corpus1,dictionary,min_topics=1,max_topics,step=1):
    kl = []
    for i in range(min_topics,max_topics,step):
        lda = models.ldamodel.LdaModel(corpus=corpus1,
            id2word=dictionary,num_topics=i)
        m1 = lda.expElogbeta
        U,cm1,V = np.linalg.svd(m1)
        #Document-topic matrix
        lda_topics = lda[my_corpus]
        m2 = matutils.corpus2dense(lda_topics, lda.num_topics).transpose()
        cm2 = l.dot(m2)
        cm2 = cm2 + 0.0001
        cm2norm = np.linalg.norm(l)
        cm2 = cm2/cm2norm
        kl.append(sym_kl(cm1,cm2))
    return kl

SyntaxError: non-default argument follows default argument (<ipython-input-8-0a369b4ed8a2>, line 2)

In [3]:
#documents
from gensim import corpora, models, similarities
num_topics=5

In [7]:
#print ([i for i in lda_corpus])
pda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=num_topics)
corpus_lda = pda[corpus]

In [1]:
pda.show_topics()

NameError: name 'pda' is not defined

In [5]:
cluster1 = [j for i,j in zip(corpus_lda,documents) if i[0][1] > i[1][1] and i[0][1] > i[2][1]]
cluster2 = [j for i,j in zip(corpus_lda,documents) if i[1][1] > i[0][1] and i[1][1] > i[2][1]]
cluster3 = [j for i,j in zip(corpus_lda,documents) if i[2][1] > i[1][1] and i[2][1] > i[0][1]]
#cluster2 = [j for i,j in zip(lda_corpus,documents) if i[1][1] > threshold]
#cluster3 = [j for i,j in zip(lda_corpus,documents) if i[2][1] > threshold]

#print (cluster1)
#print (cluster2)
#print (cluster3)
#print(len(cluster1))
#print(len(cluster2))
#print(len(cluster3))
#print(len(corpus))

In [28]:
#corpus

In [30]:
#cluster1